본 EDA 파일은 모델에 맞게 샘플링릊 ㅣㄴ행하고자 여러 가지 측면에서 데이터를 보고, 불필요한 데이터 row, col을 걸러내기 위한 작업을 진행합니다

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
path = '/home/siyun/ephemeral/data/'
df = pd.read_csv(path + 'gift_data.csv')

In [4]:
print(df['price'].mean())
print(df['price'].quantile(0.3))
df['price'].quantile(0.7)

47374.42467744867
12600.0


33000.0

product_id의 빈도가 최소 2인 것만 반환

각 카테고리1 별 카테고리2의 통계값 확인

In [3]:
def cat_stats(df) : 
    for col1 in df['category_1'].unique():
        # 특정 category_1 값에 해당하는 행들만 선택
        filtered_df1 = df[df['category_1'] == col1]
        
        # category_1에 대한 price 통계 계산
        mean_price1 = filtered_df1['price'].mean()
        std_price1 = filtered_df1['price'].std()
        
        mean_review1 = filtered_df1['num_review'].mean()
        std_review1 = filtered_df1['num_review'].std()
        
        # 결과 출력
        print(f"len : {len(filtered_df1)} || Category_1: {col1} || price_mean :{int(mean_price1)}({std_price1:.3f}) || price_min : {filtered_df1['price'].min()} || Q_0.3: {int(filtered_df1['price'].quantile(0.3))} || Q_0.5: {int(filtered_df1['price'].median())} || Q_0.99: {int(filtered_df1['price'].quantile(0.99))} || price_max : {filtered_df1['price'].max()}")

        print(f"                                        review_mean: {int(mean_review1)}({std_review1:.3f}) || review_min : {filtered_df1['price'].min()} || Q_0.3: {int(np.quantile(std_review1, 0.3))} || Q_0.5: {int(np.median(std_review1))}")
        # print(f"Q 0.3보다 낮은 row의 개수 : {len(filtered_df1[filtered_df1['price'] < filtered_df1['price'].quantile(0.3)])} 의 비율 : {len(filtered_df1[filtered_df1['price'] < filtered_df1['price'].quantile(0.3)]) / len(df)}")
        # category_2의 unique 값에 대해 반복
        for col2 in filtered_df1['category_2'].unique():
            # 특정 category_2 값에 해당하는 행들만 선택
            filtered_df2 = filtered_df1[filtered_df1['category_2'] == col2]

            if len(filtered_df2) <= 1 :
                pass
            else :
                # category_2에 대한 price 통계 계산
                mean_price2 = filtered_df2['price'].mean()
                std_price2 = filtered_df2['price'].std()
                
                
                mean_review2 = filtered_df2['num_review'].mean()
                std_review2 = filtered_df2['num_review'].std()
                
                # 각 카테고리1의 하위 카테고리인 카테고리2의 quantile 0.3 이하를 제거 

                # 결과 출력
                print(f"    len : {len(filtered_df2)} || Category_2: {col2} || Category_1: {col1} || price_mean :{int(mean_price2)}({std_price2:.3f}) || price_min : {filtered_df2['price'].min()} || Q_0.3: {int(filtered_df2['price'].quantile(0.3))} || Q_0.5: {int(filtered_df2['price'].median())} || Q_0.99: {int(filtered_df2['price'].quantile(0.99))} || price_max : {filtered_df2['price'].max()}")
                print(f"                                             review_mean: {int(mean_review1)}({std_review1:.3f}) || review_min : {filtered_df2['price'].min()} || Q_0.3: {int(np.quantile(std_review2, 0.3))} || Q_0.5: {int(np.median(std_review2))}")


filtering

가격
- 비정상적으로 높은 부분을 제거하기 위해(비정상적으로 높다 == 어뷰징의 가능성) std 약 3sigma(97%)의 상한선

리뷰
- 너무 적은 것은 표본으로써 기능을 못한다고 생각함.
- 즉, 인기가 없다 != 독특하다 가 아니라 인기가 없다 == 안사는데 이유가 있다 or 대체제로써 가치가 낮다라고 생각.
- 리뷰는 이에 대한 하나의 척도로써 주관적으로 판단함.


In [4]:
filtered_df = pd.DataFrame()

for col1 in df['category_1'].unique():
    # 특정 category_1 값에 해당하는 행들만 선택
    filtered_df1 = df[df['category_1'] == col1]
    
    # category_2의 unique 값에 대해 반복
    for col2 in filtered_df1['category_2'].unique():
        # 특정 category_2 값에 해당하는 행들만 선택
        filtered_df2 = filtered_df1[filtered_df1['category_2'] == col2]

        # 각 카테고리1의 하위 카테고리인 카테고리2의 quantile 0.3 이하를 제거 
        price_q_3 = filtered_df2['price'].quantile(0.3)
        price_q_99 =  filtered_df2['price'].quantile(0.99)
        rev_q_0_50 = filtered_df2['num_review'].quantile(0.5)
        
        filtered_q3 = filtered_df2[(filtered_df2['price'] > price_q_3) & (filtered_df2['price'] <= price_q_99) & (filtered_df2['num_review'] >= rev_q_0_50)]
        filtered_df = pd.concat([filtered_df, filtered_q3])
        

필터링한 후 확인

In [ ]:
cat_stats(filtered_df)

샘플링할 비율 계산

In [5]:
filtered_df_col = filtered_df[['product_id','category_1','category_2','category_3','rating','num_review','price']]

for col1 in filtered_df_col['category_1'].unique():
    filtered_df1 = filtered_df[filtered_df_col['category_1'] == col1]
    
    print(f" 카테고리1 [{col1}] ratio : {len(filtered_df1) / len(filtered_df_col)}")
    
    for col2 in filtered_df1['category_2'].unique():
        filtered_df2 = filtered_df1[filtered_df1['category_2'] == col2]
        print(f"\t 카테고리2 [{col2}]의 상대 ratio : {len(filtered_df2) / len(filtered_df1)}")
        


 카테고리1 [생일선물] ratio : 0.3703838868324557
	 카테고리2 [환갑/칠순선물]의 상대 ratio : 0.23199455731243843
	 카테고리2 [4050대 여성]의 상대 ratio : 0.21850513770937927
	 카테고리2 [2030대 여성]의 상대 ratio : 0.15136301787641346
	 카테고리2 [10대 여성]의 상대 ratio : 0.0803031013935157
	 카테고리2 [2030대 남성]의 상대 ratio : 0.11741660019706282
	 카테고리2 [4050대 남성]의 상대 ratio : 0.14606108947590674
	 카테고리2 [10대 남성]의 상대 ratio : 0.0254070285741097
	 카테고리2 [파티용품]의 상대 ratio : 0.025852765917515133
	 카테고리2 [e쿠폰]의 상대 ratio : 0.001994088115234833
	 카테고리2 [케이크]의 상대 ratio : 0.0011026134284239666
 카테고리1 [퇴직선물] ratio : 0.04211633039639921
	 카테고리2 [화장품]의 상대 ratio : 0.6622653187538684
	 카테고리2 [영양제]의 상대 ratio : 0.21270889209820507
	 카테고리2 [향초_디퓨저]의 상대 ratio : 0.11140911904270683
	 카테고리2 [꽃]의 상대 ratio : 0.013616670105219723
 카테고리1 [취업선물] ratio : 0.07731609405140503
	 카테고리2 [화장품]의 상대 ratio : 0.3596313778377163
	 카테고리2 [데스크테리어]의 상대 ratio : 0.23881771184535852
	 카테고리2 [영양제]의 상대 ratio : 0.3609799955046078
	 카테고리2 [디지털_가전]의 상대 ratio : 0.04057091481231737
 카테고리1 [집

In [12]:
# 카테고리1 별 비율 계산
cat1_ratios = filtered_df['category_1'].value_counts(normalize=True)

# 샘플링할 최종 데이터프레임 초기화
final_sample_df = pd.DataFrame()

# 총 샘플링할 개수
total_samples = 20000

# 카테고리1 별로 루프
for cat1, cat1_ratio in cat1_ratios.items():
    cat1_df = filtered_df[filtered_df['category_1'] == cat1]
    # print(cat1_ratio)
    
    # 카테고리1의 비율에 따라 할당할 샘플 수 계산 (1 - 카테고리1의 비율)
    cat1_sample_size = int((cat1_ratio) * total_samples)
    # print(cat1_sample_size)
    
    # 카테고리2 별 비율 계산
    cat2_ratios = cat1_df['category_2'].value_counts(normalize=True)
    
    # 카테고리2 별로 샘플링
    for cat2, cat2_ratio in cat2_ratios.items():
        cat2_df = cat1_df[cat1_df['category_2'] == cat2]
        
        # 카테고리2 별 할당할 샘플 수 (카테고리2의 비율 * 카테고리1 내 샘플 수)
        cat2_sample_size = max(3, int(cat2_ratio * cat1_sample_size)) # 적어도 3개는 샘플링
        
        # 샘플링 후 최종 데이터프레임에 추가
        sampled_df = cat2_df.sample(n=cat2_sample_size, replace=False) # replace=False로 중복 없이 추출
        final_sample_df = pd.concat([final_sample_df, sampled_df])


In [13]:
# 최종 샘플링된 데이터프레임의 길이 조정
# 필요에 따라서는 여기서 추가 조정을 할 수 있습니다. 예를 들어 total_samples보다 많이 샘플링된 경우 조정 필요
if len(final_sample_df) > total_samples:
    final_sample_df = final_sample_df.sample(n=total_samples)

비율 확인

In [14]:
cat_stats(final_sample_df)
# final_sample_df.groupby('category_1').apply(lambda x: x.groupby('category_2')['price'].min())


len : 7402 || Category_1: 생일선물 || price_mean :54977(92172.503) || price_min : 2000 || Q_0.3: 24000 || Q_0.5: 31900 || Q_0.99: 489000 || price_max : 1899000
                                        review_mean: 1926(2418.866) || review_min : 2000 || Q_0.3: 2418 || Q_0.5: 2418
    len : 1718 || Category_2: 환갑/칠순선물 || Category_1: 생일선물 || price_mean :76381(104395.273) || price_min : 19810 || Q_0.3: 29900 || Q_0.5: 39000 || Q_0.99: 579319 || price_max : 898000
                                             review_mean: 1926(2418.866) || review_min : 19810 || Q_0.3: 2444 || Q_0.5: 2444
    len : 1618 || Category_2: 4050대 여성 || Category_1: 생일선물 || price_mean :41245(31273.110) || price_min : 14950 || Q_0.3: 23808 || Q_0.5: 30000 || Q_0.99: 168659 || price_max : 230000
                                             review_mean: 1926(2418.866) || review_min : 14950 || Q_0.3: 2541 || Q_0.5: 2541
    len : 1121 || Category_2: 2030대 여성 || Category_1: 생일선물 || price_mean :43392(73831.796) || price_min : 1

최종적으로 product_id의 빈도가 3이상인 것만 반환

In [17]:
product_id_counts = final_sample_df['product_id'].value_counts()

# 빈도가 2 이상인 product_id 선택
valid_product_ids = product_id_counts[product_id_counts >= 3].index

# 해당 product_id만 포함하는 행을 필터링
final_sample_df_freq_2= final_sample_df[final_sample_df['product_id'].isin(valid_product_ids)]

In [18]:
cat_stats(final_sample_df_freq_2)

len : 1631 || Category_1: 생일선물 || price_mean :50553(66518.129) || price_min : 12900 || Q_0.3: 26400 || Q_0.5: 33900 || Q_0.99: 386000 || price_max : 879000
                                        review_mean: 2200(2601.325) || review_min : 12900 || Q_0.3: 2601 || Q_0.5: 2601
    len : 323 || Category_2: 환갑/칠순선물 || Category_1: 생일선물 || price_mean :45751(29870.281) || price_min : 19840 || Q_0.3: 28800 || Q_0.5: 35900 || Q_0.99: 156974 || price_max : 226000
                                             review_mean: 2200(2601.325) || review_min : 19840 || Q_0.3: 2816 || Q_0.5: 2816
    len : 366 || Category_2: 4050대 여성 || Category_1: 생일선물 || price_mean :43173(28912.891) || price_min : 15000 || Q_0.3: 26595 || Q_0.5: 33950 || Q_0.99: 147505 || price_max : 208600
                                             review_mean: 2200(2601.325) || review_min : 15000 || Q_0.3: 2765 || Q_0.5: 2765
    len : 230 || Category_2: 2030대 여성 || Category_1: 생일선물 || price_mean :48317(86509.783) || price_min : 1290

In [19]:
final_sample_df_freq_2['product_id'].nunique()

967

In [20]:
final_sample_df_freq_2.shape

(3167, 11)

filtered_sampling_df 저장

In [48]:
final_sample_df_freq_2.to_csv('/home/siyun/ephemeral/data/filtered_gift.csv', index=False)

In [21]:
filtered_encoding = pd.get_dummies(final_sample_df_freq_2, columns=['category_1','category_2','category_3'])

In [22]:
data_index = filtered_encoding.copy()
unique_product_ids = np.unique(data_index['product_id']).tolist()
# id2idx: 각 product_id를 연속적인 정수 인덱스로 매핑
id2idx = {product_id: idx for idx, product_id in enumerate(unique_product_ids)}

# idx2id: 연속적인 정수 인덱스를 원래의 product_id로 매핑
idx2id = {idx: product_id for product_id, idx in id2idx.items()}

In [23]:
data_index['mapped_product_id'] = data_index['product_id'].apply(lambda x: id2idx[x])
cols = list(data_index.columns)

In [24]:
features = cols.copy()

# 새로운 feature 리스트 생성
removes = ['product_id','mapped_product_id', 'product_name', 'brand', 'image_url', 'product_url']
for remove in removes : 
    features.remove(remove)
target = ['mapped_product_id']

In [25]:
from sklearn.preprocessing import LabelEncoder

X = data_index[features]
y = data_index[target]

In [26]:
X

,rating,num_review,price,category_1_개업선물,category_1_결혼기념일선물,category_1_새차선물,category_1_생일선물,category_1_집들이선물,category_1_출산선물,category_1_취업선물,...,category_3_와인용품,category_3_이어폰_헤드폰,category_3_인테리어용품,category_3_자동차용품,category_3_주얼리,category_3_주얼리/패션잡화,category_3_커피/차/와인용품,category_3_홈데코/패브릭,category_3_홈프래그런스,category_3_화장품
228153,4.8,597,21850,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10063,4.8,916,82200,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3282,4.8,5052,49000,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
176748,4.8,630,26000,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
281346,4.8,7984,21250,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241847,4.7,959,29000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
109955,4.8,3117,24000,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
67529,4.7,838,28500,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
35624,4.8,889,38400,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [32]:
y

,mapped_product_id
228153,631
10063,20
3282,7
176748,475
281346,805
...,...
241847,690
109955,313
67529,197
35624,97


In [33]:
from sklearn.model_selection import train_test_split

# train, test, valid로 나눕시다
train_x,test_x,train_y,test_y = train_test_split(X,y, test_size=0.33, random_state=42, stratify=y)

# x_train,x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.1, random_state=42)

In [34]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score, confusion_matrix
from lightgbm import LGBMClassifier
import wandb

In [ ]:
import wandb
wandb.init()

In [36]:
def train_and_evaluate(X, y, config):
    # 모델 초기화
    model = LGBMClassifier(
        objective='multiclass',  
        learning_rate=config.learning_rate,
        num_leaves=config.num_leaves,
        n_estimators=config.n_estimators,
        max_depth=config.max_depth,
        verbose=-1,
        # n_jobs= 4,
        device_type = 'cuda' 
    )
    
    # K-폴드 교차 검증 설정
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # 전체 모델 성능을 위한 리스트 초기화
    overall_accuracies = []
    overall_f1_scores = []
    overall_recalls = []
    overall_precisions = []
    overall_auc_scores = []
    
    # 레이블별 예측 결과를 저장할 리스트
    y_pred_list = []
    
    for train_index, test_index in skf.split(X, y):
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        
        # 레이블별 예측 결과를 저장할 임시 배열
        model.fit(X_train_fold, y_train_fold)
        
        y_pred_fold = model.predict(X_test_fold)
        
        # 전체 모델의 성능 계산
        overall_accuracy = accuracy_score(y_test_fold, y_pred_fold)
        overall_f1 = f1_score(y_test_fold, y_pred_fold, average='macro')
        # overall_recall = recall_score(y_test_fold, y_pred_fold.round())
        # overall_precision = precision_score(y_test_fold, y_pred_fold.round())
        # overall_auc = roc_auc_score(y_test_fold, y_pred_fold.round())
        
        overall_accuracies.append(overall_accuracy)
        overall_f1_scores.append(overall_f1)
        # overall_recalls.append(overall_recall)
        # overall_precisions.append(overall_precision)
        # overall_auc_scores.append(overall_auc)
        
        # 레이블별 예측 결과를 리스트에 추가
        y_pred_list.append(y_pred_fold)
        
    # 평균 전체 모델 성능 로깅
    wandb.log({
        "overall_accuracy": np.mean(overall_accuracies),
        "overall_f1_score": np.mean(overall_f1_scores),
        # "overall_recall": np.mean(overall_recalls),
        # "overall_precision": np.mean(overall_precisions),
        # "overall_auc_score": np.mean(overall_auc_scores)
    })

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'   
    },
    'parameters': {
        'learning_rate': {
            'min': 0.001,
            'max': 0.1
        },
        'num_leaves': {
            'values': [4,5]
        },
        'n_estimators': {
            'values': [10,20,50]
        },
        'max_depth': {
            'values': [4,8,10]
        }
    }
}

def sweep_train():
    # WandB Sweep 실행을 위한 함수
    with wandb.init() as run:
        config = wandb.config
        train_and_evaluate(X, y, config)

# Sweep 설정을 WandB에 등록하고 Sweep ID 받기
sweep_id = wandb.sweep(sweep_config, project="filtering")

# Sweep Agent 시작
wandb.agent(sweep_id, sweep_train)


Create sweep with ID: l399d4kw
Sweep URL: https://wandb.ai/tldbs5026/filtering/sweeps/l399d4kw


wandb: Agent Starting Run: 3fqj0r8y with config:
wandb: 	learning_rate: 0.08213555384427194
wandb: 	max_depth: 4
wandb: 	n_estimators: 10
wandb: 	num_leaves: 4
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


overall_accuracy,▁
overall_f1_score,▁
overall_accuracy,0.02273
overall_f1_score,0.01085


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rjdgq7qq with config:
wandb: 	learning_rate: 0.01149821336514329
wandb: 	max_depth: 4
wandb: 	n_estimators: 20
wandb: 	num_leaves: 4
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [42]:
# acc,f1 70세팅 : 0.01,4,30,8

params = {
    'learning_rate': 0.01,  # 예시 값
    'num_leaves': 4,      # 예시 값
    'n_estimators': 30,    # 예시 값
    'max_depth': 8,         # 예시 값
    'device_type' : 'cuda' 
}

# 전체 데이터에 대한 모델 학습 함수
def train_full_model(X, y, params):
    num_classes = y.nunique()
    model = LGBMClassifier(
        objective='multiclass',
        num_class=num_classes,
        learning_rate=params['learning_rate'],
        num_leaves=params['num_leaves'],
        n_estimators=params['n_estimators'],
        max_depth=params['max_depth'],
        verbose=-1,
        device_type=params['device_type']
    )
    model.fit(X, y)
    return model




In [ ]:
train_and_evaluate(train_x, train_y, params)

모델을 적용했는데, 끝나면 혹시모르니까 pkl로 저장해놓기!!!!!!!!!!!!!!!

In [43]:
model = train_full_model(train_x, train_y, params)

In [47]:
# from sklearn.externals import joblib
from lightgbm import LGBMClassifier
import pickle

params = {
    'learning_rate': 0.01,  # 예시 값
    'num_leaves': 4,      # 예시 값
    'n_estimators': 30,    # 예시 값
    'max_depth': 8,         # 예시 값
    'device_type' : 'cuda' 
}


model_save_path = "/home/siyun/ephemeral/boosting/output/lgbm_lr_0.01_lea_4_esti_30_dep_8.pkl"  # 변경된 경로
with open(model_save_path, "wb") as file:
    pickle.dump(model, file)


model_save_path  # 출력하여 저장된 경로 확인

'/home/siyun/ephemeral/boosting/output/lgbm_lr_0.01_lea_4_esti_30_dep_8.pkl'

In [ ]:
with open(model_save_path, 'rb') as file:
    loaded_model = pickle.load(file)

In [44]:
y_pred = model.predict(test_x)
# train_x,test_x,train_y,test_y

accuracy = accuracy_score(test_y, y_pred)
f1 = f1_score(test_y, y_pred, average='macro')
conf_matrix = confusion_matrix(test_y, y_pred)

In [45]:
print(accuracy)
print(f1)
print(conf_matrix)

0.7745709828393136
0.7251117068811439
[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


In [ ]:
def add_prediction_probabilities(models, X, data, target):
    # 해당 모델에 대한 확률 계산
    prob = model.predict_proba(X)
    
    # 가장 확률이 높은 클래스의 인덱스 찾기
    highest_prob_index = np.argmax(prob, axis=1)
    # 가장 확률이 높은 클래스에 대한 확률 선택
    
    # 가장 확률이 높은 클래스의 인덱스를 사용하여 실제 product_id를 매핑 (가정: model.classes_에 매핑 정보가 있음)
    predicted_product_ids = [idx2id[idx] for idx in highest_prob_index]
    
    highest_prob = np.max(prob, axis=1)
    
    # 결과를 데이터에 추가
    data['predicted_product_id'] = predicted_product_ids
    data['highest_probability'] = highest_prob
    
    return data
    
    

# train_x에 대한 inference
product_id_prob = add_prediction_probabilities(model, X, data_index.copy(), 'mapped_product_id')
product_id_prob

In [57]:
product_id_prob

,product_id,product_name,brand,rating,num_review,price,image_url,product_url,category_1_개업선물,category_1_결혼기념일선물,...,category_3_주전자/티포트,category_3_커피/차/와인용품,category_3_패션의류/잡화,category_3_패션잡화,category_3_홈데코/패브릭,category_3_홈프래그런스,category_3_화장품,mapped_product_id,predicted_product_id,highest_probability
12134,327583768,제로이드 수딩 크림 80ml[소식알림5%할인],미소드림엠디몰,4.9,1408,27000,https://shop-phinf.pstatic.net/20190329_289/md...,https://shopping.naver.com/gift/products/32758...,False,False,...,False,False,False,False,False,False,True,72,327583768,0.204500
30943,585308247,[세노비스] 눈에 좋은 루테인+오메가 3 (80캡슐/40일분) x 2개,세노비스몰,4.8,771,55400,https://shop-phinf.pstatic.net/20240110_265/17...,https://shopping.naver.com/gift/products/58530...,False,False,...,False,False,False,False,False,False,False,219,585308247,0.042827
57375,2421053713,제주 새콤달콤 천혜향 9kg,건영농장,4.8,1084,68000,https://shop-phinf.pstatic.net/20180110_54/gun...,https://shopping.naver.com/gift/products/24210...,False,False,...,False,False,False,False,False,False,False,383,2421053713,0.032955
191803,5315648099,닥터지 레드 블레미쉬 클리어 수딩 크림 70ml (튜브형) 진정 수분,고운세상 닥터지,4.8,2409,27600,https://shop-phinf.pstatic.net/20230802_246/16...,https://shopping.naver.com/gift/products/53156...,False,False,...,False,False,False,False,False,False,True,1217,5315648099,0.206639
241960,6006984143,항공직송 골드망고 태국 망고 제철 열대과일 남독마이 선물세트 2.5kg 5kg,올즈마켓,4.7,487,29900,https://shop-phinf.pstatic.net/20230912_40/169...,https://shopping.naver.com/gift/products/60069...,False,False,...,False,False,False,False,False,False,False,1578,6006984143,0.124709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309983,8356989623,커스텀프로 프리미엄 실버 현대 기아 자동차 외제차 브랜드 주문제작 듀얼 주차번호판 ...,커스텀프로,4.8,427,19900,https://shop-phinf.pstatic.net/20230406_55/168...,https://shopping.naver.com/gift/products/83569...,False,False,...,False,False,False,False,False,False,False,2103,8356989623,0.237511
93174,4220438049,아임반 자동차 타공 핸들커버 차량용 핸들커버,아임반,4.8,1157,30900,https://shop-phinf.pstatic.net/20231226_181/17...,https://shopping.naver.com/gift/products/42204...,False,False,...,False,False,False,False,False,False,False,588,4220438049,0.183994
26304,490088923,자동차 알칸타라 원형 핸들커버 얇은 차량 고급,이지스오토랩,4.8,9999,38900,https://shop-phinf.pstatic.net/20240127_194/17...,https://shopping.naver.com/gift/products/49008...,False,False,...,False,False,False,False,False,False,False,190,490088923,0.303505
41120,2007736686,알칸타라 핸들커버 D컷 자동차 차량 프리미엄 페이스리프트,이지스오토랩,4.8,640,38900,https://shop-phinf.pstatic.net/20210726_50/162...,https://shopping.naver.com/gift/products/20077...,False,False,...,False,False,False,False,False,False,False,287,2007736686,0.161918


AttributeError: 'numpy.dtype[int64]' object has no attribute 'astype'

유사도 계산

In [58]:
def find_similar_products(user_product_id, product_id_prob, n=10):
    """
    주어진 product_id에 대해 상위 n개의 가장 유사한 predicted_product_id를 찾습니다.
    
    Args:
    - user_product_id: 유저의 product_id.
    - product_id_prob: 예측된 product_id와 그 확률이 포함된 DataFrame.
    - n: 반환할 유사한 product_id의 개수.
    
    Returns:
    - 상위 n개의 가장 유사한 predicted_product_id 리스트.
    """

    if user_product_id not in product_id_prob['product_id'].values:
        print("Product ID not found in the DataFrame.")
        return []
    
    # 해당 product_id에 대한 predicted_product_id 찾기
    predicted_product_id = product_id_prob[product_id_prob['product_id'] == user_product_id]['predicted_product_id'].values[0]
    
    
    # 유사도 계산이나 로직에 따라 상위 n개의 유사한 product_id 선택
    # 여기서는 예시로, 해당 product_id의 행을 제외하고 랜덤으로 n개를 선택합니다.
    # 실제 구현에서는 유사도 계산을 통해 이를 결정해야 합니다.
    similar_products = product_id_prob[product_id_prob['product_id'] != user_product_id]['predicted_product_id'].sample(n=n).tolist()
    
    return similar_products


In [64]:
user_product_id = 585308247  # 예시 Product ID
top_similar_products = find_similar_products(user_product_id, product_id_prob, n=10)
print(top_similar_products)

[5592901100, 4705611814, 5055309685, 6683299124, 4813450144, 5050877293, 467918571, 3213177490, 9182726060, 5830123928]


In [71]:
product_id_prob.loc[product_id_prob['product_id'] ==5055309685]['image_url']

165529    https://shop-phinf.pstatic.net/20231012_251/16...
165524    https://shop-phinf.pstatic.net/20231012_251/16...
165523    https://shop-phinf.pstatic.net/20231012_251/16...
165525    https://shop-phinf.pstatic.net/20231012_251/16...
Name: image_url, dtype: object